In [2]:
import cv2
import numpy as np
import time
import random

# ============================================
# 1. Configurações e Inicializações
# ============================================

# Definir a área mínima aceitável para o convex hull
MIN_HULL_AREA = 1000  # Ajuste esse valor conforme necessário

# Caminho para o sprite
FOLDER = 'pokemon_dataset/images/'
POKEMON = 'abra'

def setup_webcam(index=1):
    """Inicializa a captura de vídeo da webcam."""
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        print("Erro ao abrir a webcam")
        exit()
    return cap

def create_trackbars():
    """Cria uma janela com trackbars para ajuste de HSV."""
    def nothing(x):
        pass
    
    cv2.namedWindow('Trackbars')
    cv2.createTrackbar('H Min', 'Trackbars', 0, 179, nothing)
    cv2.createTrackbar('H Max', 'Trackbars', 179, 179, nothing)
    cv2.createTrackbar('S Min', 'Trackbars', 0, 255, nothing)
    cv2.createTrackbar('S Max', 'Trackbars', 255, 255, nothing)
    cv2.createTrackbar('V Min', 'Trackbars', 0, 255, nothing)
    cv2.createTrackbar('V Max', 'Trackbars', 255, 255, nothing)

def load_sprite(folder, pokemon):
    """Carrega a imagem do sprite com canal alfa."""
    sprite = cv2.imread(f"{folder}{pokemon}.png", cv2.IMREAD_UNCHANGED)
    if sprite is None:
        print(f"Erro ao carregar o sprite: {folder}{pokemon}.png")
        exit()
    return sprite

# ============================================
# 2. Captura de Fundo
# ============================================

def capture_background(cap):
    """Captura uma imagem de fundo quando o usuário pressiona 's'."""
    print("Posicione sua mão fora da câmera e pressione 's' para capturar o fundo.")
    print("Pressione 'q' para sair do jogo.")
    while True:
        success, background = cap.read()
        if not success:
            print("Falha ao capturar o frame de fundo.")
            continue
        cv2.imshow('Captura de Fundo', background)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            break
        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()
    cv2.destroyWindow('Captura de Fundo')
    return background

# ============================================
# 3. Processamento de Frames
# ============================================

def get_hsv_values():
    """Obtém os valores atuais das trackbars para HSV."""
    h_min = cv2.getTrackbarPos('H Min', 'Trackbars')
    h_max = cv2.getTrackbarPos('H Max', 'Trackbars')
    s_min = cv2.getTrackbarPos('S Min', 'Trackbars')
    s_max = cv2.getTrackbarPos('S Max', 'Trackbars')
    v_min = cv2.getTrackbarPos('V Min', 'Trackbars')
    v_max = cv2.getTrackbarPos('V Max', 'Trackbars')
    return h_min, h_max, s_min, s_max, v_min, v_max

def create_skin_mask(frame, background, hsv_limits):
    """Cria uma máscara para a cor da pele baseada nos limites de HSV."""
    # Subtrair o fundo do frame atual
    fg_mask = cv2.absdiff(background, frame)
    
    # Converter a foreground mask para Grayscale para melhor visualização
    fg_mask_gray = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2GRAY)
    
    # Exibir a subtração de fundo em grayscale para fins de explicação
    cv2.imshow('Foreground Mask', fg_mask_gray)  # Janela atualizada
    
    # Converter o frame para o espaço de cor HSV
    hsv = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2HSV)
    
    # Definir os limites para a cor da pele em HSV
    lower_skin = np.array([hsv_limits[0], hsv_limits[2], hsv_limits[4]], dtype=np.uint8)
    upper_skin = np.array([hsv_limits[1], hsv_limits[3], hsv_limits[5]], dtype=np.uint8)
    
    # Criar uma máscara para a cor da pele
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Aplicar operações morfológicas para limpar a máscara
    kernel = np.ones((3, 3), np.uint8)
    skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)
    
    return skin_mask

# ============================================
# 4. Detecção de Contornos e Convex Hull
# ============================================

def find_largest_convex_hull(skin_mask):
    """Encontra o maior contorno e calcula seu convex hull."""
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None, 0, False
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_valid = hull_area >= MIN_HULL_AREA
    return hull, hull_area, hull_valid

# ============================================
# 5. Atualização do Sprite
# ============================================

def update_sprite_position(sprite_pos, sprite_speed, frame_dimensions, sprite_width, sprite_height):
    """Atualiza a posição do sprite e verifica colisões com as bordas."""
    sprite_x, sprite_y = sprite_pos
    speed_x, speed_y = sprite_speed
    sprite_x += speed_x
    sprite_y += speed_y

    frame_width, frame_height = frame_dimensions

    # Verificar colisão com as bordas
    if sprite_x < 0 or sprite_x + sprite_width > frame_width:
        speed_x *= -1
    if sprite_y < 0 or sprite_y + sprite_height > frame_height:
        speed_y *= -1

    sprite_x = max(0, min(sprite_x, frame_width - sprite_width))
    sprite_y = max(0, min(sprite_y, frame_height - sprite_height))

    return (sprite_x, sprite_y), (speed_x, speed_y)

# ============================================
# 6. Renderização
# ============================================

def render_sprite(frame, sprite, position):
    """Desenha o sprite na posição especificada com transparência."""
    sprite_x, sprite_y = position
    sprite_overlay = sprite[:, :, :3]
    alpha_mask = sprite[:, :, 3] / 255.0
    for c in range(3):
        frame[sprite_y:sprite_y + sprite.shape[0],
              sprite_x:sprite_x + sprite.shape[1], c] = \
            sprite_overlay[:, :, c] * alpha_mask + \
            frame[sprite_y:sprite_y + sprite.shape[0],
                  sprite_x:sprite_x + sprite.shape[1], c] * (1 - alpha_mask)
    return frame

# ============================================
# 7. Função Principal
# ============================================

def main():
    # Inicializações
    cap = setup_webcam()
    create_trackbars()
    sprite = load_sprite(FOLDER, POKEMON)
    frame_width, frame_height = 640, 480  # Ajuste conforme necessário
    sprite_width, sprite_height = sprite.shape[1], sprite.shape[0]
    
    # Inicializar a posição e velocidade do sprite
    sprite_x = random.randint(0, frame_width - sprite_width)
    sprite_y = random.randint(0, frame_height - sprite_height)
    sprite_speed_x = random.choice([-5, 5])
    sprite_speed_y = random.choice([-5, 5])

    score = 0
    game_started = False
    prev_time = time.time()

    # Capturar a imagem de fundo
    background = capture_background(cap)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Falha ao capturar o frame")
            break

        # Redimensionar frame se necessário
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Processamento de frame
        hsv_limits = get_hsv_values()
        skin_mask = create_skin_mask(frame, background, hsv_limits)

        # Detecção de convex hull
        hull, hull_area, hull_valid = find_largest_convex_hull(skin_mask)
        if hull_valid:
            cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

        # Estado do jogo
        if not game_started:
            cv2.putText(frame, "Aperte 'n' para iniciar o jogo", (50, 240),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow('Webcam - Convex Hull', frame)
            cv2.imshow('Webcam - Threshold', skin_mask)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('n'):
                game_started = True
            elif key == ord('q'):
                break
            continue

        # Atualizar posição do sprite
        (sprite_x, sprite_y), (sprite_speed_x, sprite_speed_y) = update_sprite_position(
            (sprite_x, sprite_y),
            (sprite_speed_x, sprite_speed_y),
            (frame_width, frame_height),
            sprite_width,
            sprite_height
        )

        # Verificar colisão com o convex hull
        if hull_valid:
            for point in hull:
                px, py = point[0]
                if sprite_x < px < sprite_x + sprite_width and sprite_y < py < sprite_y + sprite_height:
                    score += 1
                    # Resetar a posição e velocidade do sprite
                    sprite_x = random.randint(0, frame_width - sprite_width)
                    sprite_y = random.randint(0, frame_height - sprite_height)
                    sprite_speed_x = random.choice([-5, 5])
                    sprite_speed_y = random.choice([-5, 5])
                    break

        # Renderizar sprite
        frame = render_sprite(frame, sprite, (sprite_x, sprite_y))

        # Calcular FPS
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time) if (curr_time - prev_time) > 0 else 0
        prev_time = curr_time

        # Adicionar informações na tela
        cv2.putText(frame, f'FPS: {int(fps)}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        if hull_valid:
            cv2.putText(frame, f'Area: {int(hull_area)}', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Mao nao detectada ou muito pequena', (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Score: {int(score)}', (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        # Exibir frames
        cv2.imshow('Webcam - Convex Hull', frame)
        cv2.imshow('Webcam - Threshold', skin_mask)

        # Controle de saída
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Encerramento
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Posicione sua mão fora da câmera e pressione 's' para capturar o fundo.
Pressione 'q' para sair do jogo.
